In [17]:
# Imports & paths
import pandas as pd
import glob
import os
from pathlib import Path

RAW_DIR = Path("data/raw")
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_FILE = OUT_DIR / "all_products.csv"

print("RAW_DIR:", RAW_DIR.resolve())
print("OUT_FILE:", OUT_FILE.resolve())


RAW_DIR: C:\Users\shiva\OneDrive\Documents\Material\Amazon_Ecommorce_agent\notebooks\data\raw
OUT_FILE: C:\Users\shiva\OneDrive\Documents\Material\Amazon_Ecommorce_agent\notebooks\data\processed\all_products.csv


In [18]:
# List CSVs present
csv_files = sorted([p for p in RAW_DIR.rglob('*') if p.is_file() and p.suffix.lower() == '.csv'])
print(f"Found {len(csv_files)} CSV files.")
for f in csv_files[:50]:
    print("-", f.name)
if not csv_files:
    print("No CSV files found in data/raw. Please add CSV files and re-run.")
    # Redefine RAW_DIR to match the path used in pd.read_csv if no CSVs found
    if not csv_files:
        RAW_DIR = Path(r"C:\Users\shiva\OneDrive\Documents\Material\Amazon_Ecommorce_agent\data\raw")
        csv_files = sorted([p for p in RAW_DIR.rglob('*') if p.is_file() and p.suffix.lower() == '.csv'])
        print(f"After redefining RAW_DIR, found {len(csv_files)} CSV files.")
        for f in csv_files[:50]:
            print("-", f.name)
        if not csv_files:
            print("Still no CSV files found. Please check the directory path.")

Found 0 CSV files.
No CSV files found in data/raw. Please add CSV files and re-run.
After redefining RAW_DIR, found 140 CSV files.
- Air Conditioners.csv
- All Appliances.csv
- All Books.csv
- All Car and Motorbike Products.csv
- All Electronics.csv
- All English.csv
- All Exercise and Fitness.csv
- All Grocery and Gourmet Foods.csv
- All Hindi.csv
- All Home and Kitchen.csv
- All Movies and TV Shows.csv
- All Music.csv
- All Pet Supplies.csv
- All Sports Fitness and Outdoors.csv
- All Video Games.csv
- Amazon Fashion.csv
- Amazon Pharmacy.csv
- Amazon-Products.csv
- Baby Bath Skin and Grooming.csv
- Baby Fashion.csv
- Baby Products.csv
- Backpacks.csv
- Badminton.csv
- Bags and Luggage.csv
- Ballerinas.csv
- Beauty and Grooming.csv
- Bedroom Linen.csv
- Blu-ray.csv
- Camera Accessories.csv
- Cameras.csv
- Camping and Hiking.csv
- Car Accessories.csv
- Car and Bike Care.csv
- Car Electronics.csv
- Car Parts.csv
- Cardio Equipment.csv
- Casual Shoes.csv
- Childrens Books.csv
- Clothing.

In [19]:
c = pd.read_csv(r"C:\Users\shiva\OneDrive\Documents\Material\Amazon_Ecommorce_agent\data\raw\Air Conditioners.csv")

In [20]:
c.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"


In [21]:
# Helper to safely read a CSV (tries common encodings & separators)
def safe_read_csv(path):
    tries = [
        {"encoding": "utf-8", "sep": ","},
        {"encoding": "utf-8", "sep": ";"},
        {"encoding": "latin1", "sep": ","},
        {"encoding": "latin1", "sep": ";"},
    ]
    for t in tries:
        try:
            df = pd.read_csv(path, encoding=t["encoding"], sep=t["sep"], low_memory=False)
            print(f"Read {path.name} with encoding={t['encoding']} sep='{t['sep']}' shape={df.shape}")
            return df
        except Exception as e:
            # print(f"Failed {t} for {path.name}: {e}")
            continue
    # last resort: let pandas infer
    df = pd.read_csv(path, low_memory=False)
    print(f"Read {path.name} with default pandas reader shape={df.shape}")
    return df


In [22]:
# Read all CSVs into list of DataFrames, attach source_file
dfs = []
for f in csv_files:
    try:
        df = safe_read_csv(f)
        df["source_file"] = f.name
        dfs.append(df)
    except Exception as e:
        print("Skipping file due to error:", f.name, e)

print("Loaded dataframes:", len(dfs))
# Show column name variety (top 100 unique column names)
cols = sorted({c.strip() for df in dfs for c in df.columns})
print("Unique columns found (sample):", cols[:100])


Read Air Conditioners.csv with encoding=utf-8 sep=',' shape=(720, 9)
Read All Appliances.csv with encoding=utf-8 sep=',' shape=(9576, 9)
Read All Books.csv with encoding=utf-8 sep=',' shape=(0, 9)
Read All Car and Motorbike Products.csv with encoding=utf-8 sep=',' shape=(1272, 9)
Read All Electronics.csv with encoding=utf-8 sep=',' shape=(9600, 9)
Read All English.csv with encoding=utf-8 sep=',' shape=(0, 9)
Read All Exercise and Fitness.csv with encoding=utf-8 sep=',' shape=(1176, 9)
Read All Grocery and Gourmet Foods.csv with encoding=utf-8 sep=',' shape=(960, 9)
Read All Hindi.csv with encoding=utf-8 sep=',' shape=(0, 9)
Read All Home and Kitchen.csv with encoding=utf-8 sep=',' shape=(1224, 9)
Read All Movies and TV Shows.csv with encoding=utf-8 sep=',' shape=(0, 9)
Read All Music.csv with encoding=utf-8 sep=',' shape=(0, 9)
Read All Pet Supplies.csv with encoding=utf-8 sep=',' shape=(648, 9)
Read All Sports Fitness and Outdoors.csv with encoding=utf-8 sep=',' shape=(1224, 9)
Read A

In [23]:
# Normalize column names to simple lower-case keys without spaces
def normalize_columns(df):
    df = df.copy()
    mapping = {c: c.strip().lower().replace(" ", "_") for c in df.columns}
    df = df.rename(columns=mapping)
    return df

dfs = [normalize_columns(df) for df in dfs]


In [24]:
c.shape

(720, 9)

In [25]:
type(dfs)

list

In [26]:
df.shape

(1104, 10)

In [27]:
type(dfs)

list

In [28]:
# Concatenate all DataFrames in dfs into a single DataFrame
all_df = pd.concat(dfs, ignore_index=True)
print("Combined DataFrame shape:", all_df.shape)
all_df.head()

Combined DataFrame shape: (1103170, 11)


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,source_file,unnamed:_0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",Air Conditioners.csv,NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",Air Conditioners.csv,NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",Air Conditioners.csv,NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",Air Conditioners.csv,NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",Air Conditioners.csv,NaN


In [29]:
all_df.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price', 'source_file',
       'unnamed:_0'],
      dtype='object')

In [32]:
all_df.isna().sum()

name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings           351588
no_of_ratings     351588
discount_price    122326
actual_price       35626
source_file            0
unnamed:_0        551585
dtype: int64

In [36]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   source_file     1103170 non-null  object 
 10  unnamed:_0      551585 non-null   float64
dtypes: float64(1), object(10)
memory usage: 92.6+ MB


In [33]:
all_df.shape

(1103170, 11)

In [34]:
all_df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,source_file,unnamed:_0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",Air Conditioners.csv,NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",Air Conditioners.csv,NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",Air Conditioners.csv,NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",Air Conditioners.csv,NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",Air Conditioners.csv,NaN


In [37]:
all_df['unnamed:_0'].value_counts()

unnamed:_0
0.0        112
1.0        112
2.0        112
3.0        112
4.0        112
          ... 
19195.0      6
19196.0      6
19197.0      6
19198.0      6
19199.0      6
Name: count, Length: 19200, dtype: int64

In [38]:
all_df.drop(columns=["unnamed:_0"], inplace=True, errors='ignore')


Extract company (brand) name from product titles

Most Amazon titles start with the brand or company name like
"Lloyd 1.5 Ton 3 Star..." → company = Lloyd.

Here’s a simple extractor (and it works quite well for structured product titles):

In [39]:
import re

def extract_company(name):
    if not isinstance(name, str) or not name.strip():
        return None
    name = name.strip()
    # Take the first word (or first two if it's like 'HP Laptop' or 'LG Refrigerator')
    parts = name.split()
    if len(parts) == 1:
        return parts[0]
    # Handle short brand names like 'LG', 'HP', 'MI'
    if len(parts[0]) <= 3 or parts[0].isupper():
        return parts[0]
    return parts[0].capitalize()

all_df["company"] = all_df["name"].apply(extract_company)


In [44]:
all_df['company'].value_counts()

company
PC            12812
Puma           9942
Shopnet        8518
U.S.           7802
Amazon         7318
              ...  
J.E               2
UDYE              2
Piestar           2
Specimen-x        2
TULOO             2
Name: count, Length: 48705, dtype: int64

In [45]:

all_df[all_df['company'] == "PC"]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,source_file,company
12975,PC SQUARE Laptop Tabletop Stand/ Computer Tabl...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PC-SQUARE-Adjustable-Erg...,4.2,"2,921",₹485,"₹1,299",All Electronics.csv,PC
19698,PC 2.1 A 2 USB Wall Socket Charger Power Panel...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51xm2mQfLQ...,https://www.amazon.in/Generic-Socket-Charger-R...,3.9,"1,045",₹384,₹502,All Electronics.csv,PC
23608,PC SQUARE Laptop Tabletop Stand/ Computer Tabl...,home & kitchen,All Home & Kitchen,https://m.media-amazon.com/images/I/61hpz-rwvl...,https://www.amazon.in/PC-SQUARE-Adjustable-Erg...,4.2,"2,923",₹485,"₹1,299",All Home and Kitchen.csv,PC
41727,PC SQUARE Laptop Tabletop Stand/ Computer Tabl...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PC-SQUARE-Adjustable-Erg...,4.2,"2,921",₹485,"₹1,299",Amazon-Products.csv,PC
48450,PC 2.1 A 2 USB Wall Socket Charger Power Panel...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51xm2mQfLQ...,https://www.amazon.in/Generic-Socket-Charger-R...,3.9,"1,045",₹384,₹502,Amazon-Products.csv,PC
...,...,...,...,...,...,...,...,...,...,...,...
936557,PC Jeweller 2 Gram 24K (995) Yellow Gold Flora...,stores,Men's Fashion,https://m.media-amazon.com/images/I/61we6Ii9+U...,https://www.amazon.in/PC-Jeweller-24KT-Yellow-...,4.0,2,"₹13,019","₹14,210",Mens Fashion.csv,PC
938259,PC Jeweller 999 Purity 10 g Queen Victoria Sil...,stores,Men's Fashion,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PC-Jeweller-Purity-Victo...,4.3,56,₹865,"₹1,000",Mens Fashion.csv,PC
1029813,PC Star Stylish Sunglasses For Mens And Womens,accessories,Sunglasses,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PC-Star-Protected-OverSi...,NaN,NaN,₹239,₹999,Sunglasses.csv,PC
1083201,PC OSWAL Women's Cotton Printed Half Sleeve Lo...,women's clothing,Western Wear,https://m.media-amazon.com/images/I/71kAWVc+1e...,https://www.amazon.in/PC-OSWAL-Womens-Printed-...,3.7,18,₹514,₹999,Western Wear.csv,PC


Clean price columns first

In [46]:
def clean_price(val):
    if pd.isna(val):
        return None
    val = str(val)
    val = re.sub(r"[^\d]", "", val)
    if val == "":
        return None
    return float(val)

all_df["discount_price"] = all_df["discount_price"].apply(clean_price)
all_df["actual_price"] = all_df["actual_price"].apply(clean_price)


In [53]:
# Correct method name: isna (or isnull)
all_df[['discount_price', 'actual_price']].isna().sum()

discount_price    122326
actual_price       35626
dtype: int64

In [54]:
price_medians = all_df.groupby(["company", "sub_category"])[["discount_price", "actual_price"]].median()

In [58]:
price_medians.index.get_level_values('company').unique()

Index(['"', '"7"', '"A"', '"DN', '"GOMUKH”', '"INTERN', '"PH"', '"SRS',
       '"TGC"', '"guruji',
       ...
       '‎‎PBC', '•', '☆LBY', '☆lby®-woven', '【2', '【desview', '【official】',
       '﻿﻿lifelong', '𝟐𝟎𝟐𝟐', '🌿ecofrolic'],
      dtype='object', name='company', length=48705)

In [59]:
def fill_prices(row):
    if pd.isna(row["actual_price"]) or pd.isna(row["discount_price"]):
        try:
            med = price_medians.loc[(row["company"], row["sub_category"])]
            if pd.isna(row["discount_price"]):
                row["discount_price"] = med["discount_price"]
            if pd.isna(row["actual_price"]):
                row["actual_price"] = med["actual_price"]
        except KeyError:
            # fallback: use sub_category median
            sub_med = all_df.groupby("sub_category")[["discount_price", "actual_price"]].median().loc[row["sub_category"]]
            if pd.isna(row["discount_price"]):
                row["discount_price"] = sub_med["discount_price"]
            if pd.isna(row["actual_price"]):
                row["actual_price"] = sub_med["actual_price"]
    return row

all_df = all_df.apply(fill_prices, axis=1)

In [60]:
# Correct method name: isna (or isnull)
all_df[['discount_price', 'actual_price']].isna().sum()

discount_price    28840
actual_price      11144
dtype: int64

Handle missing ratings and no_of_ratings

You’re 100% correct — naive median fill per sub_category doesn’t capture real user engagement.
Instead:

If a brand has other products in the same sub_category → take their median rating.

Otherwise, fallback to sub_category median.

In [63]:
all_df["no_of_ratings"] = (
    all_df["no_of_ratings"]
    .astype(str)
    .str.replace(",", "")
)
all_df["no_of_ratings"] = pd.to_numeric(all_df["no_of_ratings"], errors="coerce")


In [64]:
all_df["ratings"] = pd.to_numeric(all_df["ratings"], errors="coerce")

In [65]:
# Compute medians
rating_medians = all_df.groupby(["company", "sub_category"])[["ratings", "no_of_ratings"]].median()

def fill_ratings(row):
    if pd.isna(row["ratings"]) or pd.isna(row["no_of_ratings"]):
        try:
            med = rating_medians.loc[(row["company"], row["sub_category"])]
            if pd.isna(row["ratings"]):
                row["ratings"] = med["ratings"]
            if pd.isna(row["no_of_ratings"]):
                row["no_of_ratings"] = med["no_of_ratings"]
        except KeyError:
            sub_med = all_df.groupby("sub_category")[["ratings", "no_of_ratings"]].median().loc[row["sub_category"]]
            if pd.isna(row["ratings"]):
                row["ratings"] = sub_med["ratings"]
            if pd.isna(row["no_of_ratings"]):
                row["no_of_ratings"] = sub_med["no_of_ratings"]
    return row

all_df = all_df.apply(fill_ratings, axis=1)

In [66]:
# Drop exact duplicate products
all_df.drop_duplicates(subset=["name", "link"], inplace=True)

# Check nulls again
print(all_df.isnull().sum())

name                  0
main_category         0
sub_category          0
image                 0
link                  0
ratings           54536
no_of_ratings     54536
discount_price    14420
actual_price       5572
source_file           0
company               0
dtype: int64


Price columns (discount_price, actual_price)

< 2 % missing is tiny.

Imputing from subcategory median is safe — pricing within subcategory is consistent (e.g. “Air Conditioners” have similar ranges).

✅ Do: fill remaining missing prices using subcategory median

In [67]:
sub_price_median = all_df.groupby("sub_category")[["discount_price", "actual_price"]].median()

def fill_remaining_prices(row):
    if pd.isna(row["discount_price"]) or pd.isna(row["actual_price"]):
        try:
            sub_med = sub_price_median.loc[row["sub_category"]]
            if pd.isna(row["discount_price"]):
                row["discount_price"] = sub_med["discount_price"]
            if pd.isna(row["actual_price"]):
                row["actual_price"] = sub_med["actual_price"]
        except KeyError:
            pass
    return row

all_df = all_df.apply(fill_remaining_prices, axis=1)


Rating & no_of_ratings

~5 % missing: a bit higher, but still manageable.

Imputing numerical ratings (like 4.1, 4.3) using medians by subcategory is fine.

no_of_ratings can vary widely — but it’s not directly used in embeddings, more for metadata/UI.
So approximate median per subcategory is acceptable.

✅ Do: fill remaining missing values with subcategory median.

In [68]:
sub_rating_median = all_df.groupby("sub_category")[["ratings", "no_of_ratings"]].median()

def fill_remaining_ratings(row):
    if pd.isna(row["ratings"]) or pd.isna(row["no_of_ratings"]):
        try:
            sub_med = sub_rating_median.loc[row["sub_category"]]
            if pd.isna(row["ratings"]):
                row["ratings"] = sub_med["ratings"]
            if pd.isna(row["no_of_ratings"]):
                row["no_of_ratings"] = sub_med["no_of_ratings"]
        except KeyError:
            pass
    return row

all_df = all_df.apply(fill_remaining_ratings, axis=1)


In [69]:
print(all_df.isnull().sum())
print("Final shape:", all_df.shape)


name              0
main_category     0
sub_category      0
image             0
link              0
ratings           0
no_of_ratings     0
discount_price    0
actual_price      0
source_file       0
company           0
dtype: int64
Final shape: (551585, 11)


In [72]:
# Save final cleaned dataset
import os
os.makedirs("data/processed", exist_ok=True)
all_df.to_csv("data/processed/all_products_cleaned.csv", index=False)

NameError: name 'all_df' is not defined